In [1]:
import os

os.chdir("../")  # Укажите путь к корню проекта
print(os.getcwd())  # Проверьте, что путь изменился

c:\Users\MishaV\Desktop\karpov\ML\L22


In [2]:
import warnings
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.pipeline import Pipeline
from sqlalchemy import create_engine
from src.data import (
    select_data_csv,
    CustomTransformer,
    train_test_split,
    transform_data,
    OHE_COLS,
    MTE_COLS,
)
from src.train import save_model, load_model, model_score

warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
engine = create_engine(
    "postgresql://robot-startml-ro:pheiph0hahj1Vaif@"
    "postgres.lab.karpov.courses:6432/startml"
)

In [3]:
# data = select_data(
#     seed=0.42,
#     count=4_800_000,
#     method="stratified",
#     users_scv=True,
#     posts_csv=True,
#     feed_data_csv=True,
#     save_path=r".\data\raw\stratified42.csv",
# )
data = pd.read_csv(r".\data\raw\stratified42.csv", parse_dates=["timestamp"])
data["timestamp"].is_monotonic_increasing

True

In [4]:
print("target с значением 1: ", (data["target"] == 1).sum())
print("target с значением 0: ", (data["target"] == 0).sum())

target с значением 1:  589259
target с значением 0:  4369645


In [5]:
X = data.drop("target", axis=1)
y = data["target"]

# более новые данные (по timestamp) относим к тестовой выборке
X_train, X_test, y_train, y_test = train_test_split(X, y, 0.2)
custom_cv = [(X_train.index, X_test.index)]

CatBoost

In [6]:
pipe = Pipeline(
    [
        ("custom_transformer", CustomTransformer()),
        (
            "model",
            CatBoostClassifier(
                random_state=42,
                auto_class_weights="Balanced",
                cat_features=OHE_COLS + MTE_COLS,
            ),
        ),
    ]
)

In [7]:
pipe.fit(X, y)

Learning rate set to 0.38982
0:	learn: 0.6802538	total: 2.08s	remaining: 34m 41s
1:	learn: 0.6750984	total: 3.89s	remaining: 32m 19s
2:	learn: 0.6730014	total: 5.67s	remaining: 31m 25s
3:	learn: 0.6720270	total: 7.62s	remaining: 31m 36s
4:	learn: 0.6712882	total: 9.34s	remaining: 30m 59s
5:	learn: 0.6658919	total: 11.2s	remaining: 30m 57s
6:	learn: 0.6653293	total: 13.2s	remaining: 31m 16s
7:	learn: 0.6632609	total: 15.3s	remaining: 31m 38s
8:	learn: 0.6611245	total: 16.9s	remaining: 30m 56s
9:	learn: 0.6607675	total: 18.9s	remaining: 31m 13s
10:	learn: 0.6607347	total: 19.9s	remaining: 29m 45s
11:	learn: 0.6602188	total: 21.7s	remaining: 29m 44s
12:	learn: 0.6599168	total: 23.9s	remaining: 30m 11s
13:	learn: 0.6591367	total: 25.6s	remaining: 30m
14:	learn: 0.6591197	total: 26.4s	remaining: 28m 51s
15:	learn: 0.6588425	total: 28.6s	remaining: 29m 19s
16:	learn: 0.6585235	total: 30.2s	remaining: 29m 6s
17:	learn: 0.6583903	total: 32.4s	remaining: 29m 26s
18:	learn: 0.6581832	total: 34.2

Pipeline(steps=[('custom_transformer',
                 CustomTransformer(drop_cols=['text', 'user_id', 'post_id', 'timestamp', 'country', 'action'])),
                ('model',
                 <catboost.core.CatBoostClassifier object at 0x0000013DC3B537F0>)])

In [8]:
y_proba = pipe.predict_proba(X_test)[:, 1]

In [9]:
model_score(X_test, y_test, y_proba)

roc_auc_score: 0.6205560268403599
average_precision_score: 0.17157966201713198
hitrate: 0.46403422582494064


In [10]:
save_model("catboost", pipe.named_steps["model"], method="catboost")

In [12]:
model = load_model("main.cbm", method="catboost")

In [13]:
pipe.set_params(model=model)

Pipeline(steps=[('custom_transformer',
                 CustomTransformer(drop_cols=['text', 'user_id', 'post_id', 'timestamp', 'country', 'action'])),
                ('model',
                 <catboost.core.CatBoostClassifier object at 0x0000013DFFB95C40>)])

In [14]:
y_proba = pipe.predict_proba(X_test)[:, 1]

In [15]:
model_score(X_test, y_test, y_proba)

roc_auc_score: 0.6205560268403599
average_precision_score: 0.17157966201713198
hitrate: 0.46403422582494064
